In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

In [12]:
data= pd.read_csv('10Switch.txt',delimiter=' ')
data

,Size,RttMin,RttAvg,RttMax
0,16,0.972,1.016,1.049
1,116,1.521,1.624,1.712
2,216,2.228,2.311,2.329
3,316,2.898,2.939,2.971
4,416,3.566,3.619,3.646
...,...,...,...,...
126,9600,24.109,24.138,24.151
127,9700,24.303,24.322,24.351
128,9800,24.301,24.398,24.481
129,9900,24.459,24.579,24.660


In [13]:
maxSize = 1472
L2size  = 1538
headers = 58

def getLenght(s):
    mod = (s%maxSize)
    a = (s // maxSize) * L2size
    if (mod != 0):
        a = a + mod + getPadding(mod) + headers
    return a

def getPadding(x):
    if (x < 18): return 18-x
    return 0

def getRttTheory(d, C):
    res= 2*d*8*(10**3)/(C)
    return res

def getRttTheory_Switch(d, C):
    if(d<=1538):
        result = 2*getRttTheory(d,C)
    else:
        result = getRttTheory(d,C) + getRttTheory(1538,C) 
    
    return result

    
getLenght  = np.vectorize(getLenght)
getPadding = np.vectorize(getPadding)
getRttTheory_Switch= np.vectorize(getRttTheory_Switch)

In [14]:
Size = data['Size'].to_numpy()

RttMin = data['RttMin'].to_numpy()
RttAvg = data['RttAvg'].to_numpy()
RttMax = data['RttMax'].to_numpy()

lenght = getLenght(Size)
RttTheorySwitch = getRttTheory_Switch(lenght, 10*10**6)

In [15]:
fig = px.line(x=lenght, y=[RttMin, RttAvg, RttMax, RttTheorySwitch],
                markers=True ,labels=dict(x="Data [bytes]", value="RTT [ms]", variable="Tipologia di RTT"),
                width=1400, height= 900)

newnamesFig = {'wide_variable_0':'RttMin', 'wide_variable_1':'RttAvg', 'wide_variable_2':'RttMax', 'wide_variable_3': 'RttTheorySwitch'}
fig.for_each_trace(lambda t: t.update(name = newnamesFig[t.name],
                                      legendgroup = newnamesFig[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnamesFig[t.name])
                                     )
                  )
fig.show()

In [16]:
def getCapacity_sec(d, rtt):
    x=  2*8*d/rtt/1000
    return x

# 2.4 nel caso di 10Mbit/s, altrimenti 0.24 per 100Mbit/s

def getCapacity_sec_TheorySwitch(d, rtt):
    if(d<=1538):
        return 2*getCapacity_sec(d, rtt)
    return 2*8*d/(rtt -2.4)/1000
    #return 2*8*d/(rtt -0.24)/1000

getCapacity_sec_TheorySwitch= np.vectorize(getCapacity_sec_TheorySwitch)

Capacity= getCapacity_sec(lenght, RttMin)
CapacityTheory = getCapacity_sec(lenght, RttTheorySwitch)

fig2 = px.line(x=lenght, y=[Capacity, CapacityTheory], labels=dict(x="Data [bytes]", value="Capacity [Mbit/s]", variable="Types of Capacity"))

newnamesFig2 = {'wide_variable_0':'Capacity(RTTmin)', 'wide_variable_1':'Capacity(RttTheory)'}
fig2.for_each_trace(lambda t: t.update(name = newnamesFig2[t.name],
                                      legendgroup = newnamesFig2[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnamesFig2[t.name])
                                     )
                  )

fig2.show()